# Generates crops ad pickle file using dataset_gen_pipe

This notebook crops images according to bounding box coordinates (one per sulcus)

# Imports

In [36]:
import sys
import os
import glob
import json
import inspect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the aims module
from soma import aims
# the brainplot package
import colorado as cld

print((sys.version))

3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]


In [37]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)


{'xvfb': <subprocess.Popen object at 0x7f15fc0bcc50>, 'original_display': ':1', 'display': 17, 'glx': 2, 'virtualgl': None, 'headless': True, 'mesa': False, 'qtapp': None}


In [38]:
# Uncomment the following line if you have not installed deep_folding with pip install -e .
# sys.path.append((os.path.abspath('../')))
import deep_folding
print(inspect.getfile(deep_folding))

/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/deep_folding/__init__.py


In [39]:
_ALL_SUBJECTS = -1

# User-specific variables

In [40]:
sulcus = 'S.T.s.ter.asc.ant.'

In [41]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [42]:
src_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

src_dir = /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/source/unsupervised


The target directory tgt_dir is where the files will be saved

In [43]:
tgt_dir = os.path.join(os.getcwd(), '../data/target')
tgt_dir = os.path.abspath(tgt_dir)
print(("tgt_dir = " + tgt_dir))

tgt_dir = /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target


In [44]:
bbox_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
bbox_dir = os.path.abspath(bbox_dir)
print(("bbox_dir = " + bbox_dir))

bbox_dir = /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/reference/bbox


In [45]:
mask_dir = os.path.join(os.getcwd(), '../data/reference/mask')
mask_dir = os.path.abspath(mask_dir)
print(("mask_dir = " + mask_dir))

mask_dir = /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/reference/mask


In [46]:
print((sys.argv))

['/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py', '-f', '/casa/home/.local/share/jupyter/runtime/kernel-5c466a5c-c25a-493a-9804-9b9f2dbf5287.json']


# Illustration of main program uses

We will first use the program with no effect by using number of subjects set to 0, or by calling the help function

### Using external calls

In [47]:
!python ../deep_folding/anatomist_tools/dataset_gen_pipe.py -n 0 -t tgt_local_dir

In [48]:
# Clean
!rm -rf tgt_local_dir

In [49]:
!python ../deep_folding/anatomist_tools/crop_definition.py --help

usage: mask.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t BBOX_DIR] [-m MASK_DIR]
               [-u SULCUS] [-i SIDE] [-p PATH_TO_GRAPH] [-n NB_SUBJECTS]
               [-v OUT_VOXEL_SIZE]

Computes mask and bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/dico/data/bv_databases/human/pclean/all
  -t BBOX_DIR, --bbox_dir BBOX_DIR
                        Target directory where to store the output bbox json
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test/bbox
  -m MASK_DIR, --mask_dir MASK_DIR
                        Target directory where to store the out

### By using the main function call

In [50]:
from deep_folding.anatomist_tools import dataset_gen_pipe
print((dataset_gen_pipe.__file__))

/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/deep_folding/anatomist_tools/dataset_gen_pipe.py


In [51]:
args = "-n 0 -t " + tgt_dir
argv = args.split(' ')

In [52]:
dataset_gen_pipe.main(argv)

In [53]:
args = "--help"
argv = args.split(' ')

In [54]:
dataset_gen_pipe.main(argv)

usage: dataset_gen_pipe.py [-h] [-s SRC_DIR] [-t TGT_DIR] [-a MASK_DIR]
                           [-b BBOX_DIR] [-m MORPHOLOGIST_DIR]
                           [-u SULCUS [SULCUS ...]] [-i SIDE] [-n NB_SUBJECTS]
                           [-e INTERP] [-p RESAMPLING] [-c CROPPING]
                           [-v OUT_VOXEL_SIZE [OUT_VOXEL_SIZE ...]]
                           [-o COMBINE_TYPE]

Generates cropped and pickle files

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR, --src_dir SRC_DIR
                        Source directory where the MRI data lies. Default is :
                        /neurospin/hcp
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the cropped and pickle
                        files. Default is :
                        /neurospin/dico/data/deep_folding/test
  -a MASK_DIR, --mask_dir MASK_DIR
                        masking directory where mask has been stored. Default
          

### By using the API function call

In [55]:
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir,
                                  bbox_dir=bbox_dir,
                                  cropping='bbox',
                                  list_sulci=sulcus,
                                  side=side,
                                  number_subjects=0)

In [56]:
def are_arrays_almost_equal(arr1, arr2, epsilon, max_number_different_pixels):
    """Returns True if at most max_number_different_pixels pixels of arrays arr1 and arr2 
    differ by more than epsilon
    
    """
    difference = (abs(arr1-arr2) >= epsilon)
    number_different_pixels = np.count_nonzero(difference)
    return number_different_pixels <= max_number_different_pixels, number_different_pixels

# Crops with mask and with nearest-neighbour interpolation

## Main program

In [57]:
tgt_dir_nearest = os.path.join(os.getcwd(), '../data/target/crops/nearest')
tgt_dir_nearest = os.path.abspath(tgt_dir_nearest)
print(("tgt_dir = " + tgt_dir_nearest))

tgt_dir = /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target/crops/nearest


In [58]:
src_dir

'/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/source/unsupervised'

In [59]:
interp = 'nearest'
dataset_gen_pipe.dataset_gen_pipe(src_dir=src_dir,
                                  tgt_dir=tgt_dir_nearest,
                                  mask_dir=mask_dir,
                                  cropping="mask",
                                  list_sulci=sulcus,
                                  side=side,
                                  interp=interp,
                                  number_subjects=1)

list_subjects =  ['100206']


## Result analysis

### Analysis of the inputs

In [60]:
# Gets source file as numpy array
skeleton_dir = os.path.join(src_dir, "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation")
vol_source_file = glob.glob(skeleton_dir + '/' + side + '*.nii.gz')[0]
vol_source = aims.read(vol_source_file)
arr_source = vol_source.arraydata()
print("shape of source skeleton = ", arr_source.shape)

shape of source skeleton =  (1, 260, 311, 260)


In [61]:
np.unique(arr_source)

array([ 0, 10, 11, 30, 40, 60, 80], dtype=int16)

In [62]:
pd.value_counts(np.resize(arr_source, arr_source.size))

11    18832738
0      2012736
60      163069
30        9634
80        5330
40          87
10           6
dtype: int64

In [63]:
mask_file = glob.glob(mask_dir + '/' + side + '/*.nii.gz')[0]
vol_mask = aims.read(mask_file)
arr_mask = vol_mask.arraydata()
print("shape of mask = ", arr_mask.shape)

shape of mask =  (1, 193, 229, 193)


In [64]:
np.unique(arr_mask)

array([0., 1., 2.], dtype=float32)

### Analysis of the outputs

Prints the list of files of the target directory

In [65]:
print("Files in crops target directory:")
print(tgt_dir_nearest)
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops target directory:
/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target/crops/nearest
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [66]:
tgt_json_file = glob.glob(tgt_dir_nearest + '/*.json')[0]
print("tgt_json_file = ", tgt_json_file, '\n')
with open(os.path.join(tgt_dir_nearest, tgt_json_file), 'r') as f:
    data_tgt = json.load(f)
    print((json.dumps(data_tgt, sort_keys=True, indent=4)))

tgt_json_file =  /host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target/crops/nearest/Ldataset.json 

{
    "bbmax": [
        156,
        170,
        109,
        1
    ],
    "bbmin": [
        130,
        151,
        61,
        0
    ],
    "bbox_dir": "/neurospin/dico/data/deep_folding/current/bbox",
    "combine_type": false,
    "cropped_dir": "/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target/crops/nearest/Lcrops",
    "date": "2022-01-12 15:39:25",
    "git_sha": "9ffedd2c4ad4d6c51ff726d7bf9940fa22e2d321",
    "interp": "nearest",
    "is_git": true,
    "list_sulci": [
        "S.T.s.ter.asc.ant._left"
    ],
    "mask_dir": "/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/reference/mask",
    "nb_subjects": 1,
    "out_voxel_size": [
        1,
        1,
        1
    ],
    "repo_working_dir": "/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding",
    "resamp

Obtained output (we read the cropped file from the target directory):

In [67]:
# Gets target crop as numpy array
cropped_target_dir = os.path.join(tgt_dir_nearest, side+'crops')
vol_target_file = glob.glob(cropped_target_dir + '/' + '*.nii.gz')
vol_target = aims.read(vol_target_file[0])
arr_target = vol_target.arraydata()
print("shape of target cropped image = ", arr_target.shape)

shape of target cropped image =  (1, 49, 20, 27)


In [68]:
np.unique(arr_target)

array([ 0, 30, 60], dtype=int16)

The scope here is to compare the different numbers present on the target array and on the source array:

In [69]:
np.around(pd.value_counts(np.resize(arr_target, arr_target.size))/arr_target.size*100, 1)

0     99.9
60     0.1
30     0.0
dtype: float64

In [70]:
np.around(pd.value_counts(np.resize(arr_source, arr_source.size))/arr_source.size*100, 1)

11    89.6
0      9.6
60     0.8
30     0.0
80     0.0
40     0.0
10     0.0
dtype: float64

### Visualization

In [71]:
print("Files in crops nearest target directory:")
print(('\n'.join(os.listdir(tgt_dir_nearest + '/' + side + 'crops'))))

Files in crops nearest target directory:
100206_normalized.nii.gz
100206_normalized.nii.gz.minf


In [72]:
target_file_dir = tgt_dir_nearest + '/' + side + 'crops'
target_file_nearest = glob.glob(target_file_dir + "/*.nii.gz")[0]
print(target_file_nearest)

/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/target/crops/nearest/Lcrops/100206_normalized.nii.gz


In [73]:
# load source skeleton data (the SliceableObject)
object_anat = a.loadObject(target_file_nearest)

# create an Axial window in anatomist
w = a.createWindow('Axial')
w.addObjects(object_anat)

AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)

In [75]:
print(vol_source_file)
# load source skeleton data (the SliceableObject)
a_source = a.loadObject(vol_source_file)
print(a_source)
# create an Axial window in anatomist
w1 = a.createWindow("Axial")
w1.addObjects(a_source)

/host/volatile/jc225751/Runs/30_clean_deep_folding/Program/deep_folding/data/source/unsupervised/ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition/default_analysis/segmentation/Lskeleton_100206.nii.gz


AnatomistInteractiveWidget(height=479, layout=Layout(height='auto', width='auto'), width=424)